In [21]:
# imports 
import numpy as np
import pandas as pd
import warnings
import joblib
import torch
from torch import nn

In [22]:
# define the path where the data is persisted
STORAGE_PATH = "../../persisted_data/feather/{}_normalized.feather"
STORAGE_PATH_MODELS = "../../persisted_data/models/{}"

In [23]:
def load_data_set(name):
    data = pd.read_feather(STORAGE_PATH.format(name))
    return data

In [24]:
value_stock = load_data_set("IBM")

In [25]:
# find all indicator columns
all_indicators = value_stock.columns[~value_stock.columns.str.contains("future|current", regex=True)]

# Trading strategies
A trading strategy or rule, as it is implemented here, is a rule assigning a desired position to every time point for each stock. The trading rule can take advantage of any indicator evaluated until the given time point.

## The regulating speculator
This strategy uses the position of the current price within a range defined by the bollinger bands, support and resistance lines or regression lines. The position is chosen to be higher, when the price is low in comparison to the range defined by the indicators.

In [2]:
# abstract definition of a regulating speculator
# the trading rule is constructed using the indicators, which are considered
def regulating_speculator(stock, indicators, min_position=0, max_position=1):
    # a nanmean of an empty slice triggers a warning, but returns nan, which is correct
    warnings.simplefilter("ignore", category=RuntimeWarning)
    # get the indicator data
    indicator_values = stock[indicators]
    
    # calculate the mean, if possible
    mean_features = np.nanmean(indicator_values, axis=1)
    
    # the rule causes a higher position in the stock, if the indicators are low (potentially underpriced)
    rule = (1 - np.where(np.isnan(mean_features), 0, mean_features))
    
    # clip the resulting rule 
    return np.clip(rule, min_position, max_position)

In [3]:
# define the indicators used for the different strategies
bollinger_indicators = indicators_bollinger = ["bollinger_position20_2", "bollinger_position50_2", "bollinger_position100_2", "bollinger_position200_2"]
horizontal_indicators = ["horizontal_position20", "horizontal_position50", "horizontal_position100", "horizontal_position200"]
regression_indicators = ["regression_position20", "regression_position50", "regression_position100", "regression_position200"]
mixed_indicators = ["bollinger_position50_2", "horizontal_position100", "regression_position100"]

# define the regulating strategies
regulating_speculator_strategies = {
    "bollinger speculator": lambda stock: regulating_speculator(stock, bollinger_indicators, 0, 1),
    "support resistance speculator": lambda stock: regulating_speculator(stock, horizontal_indicators, 0, 1),
    "regression position speculator": lambda stock: regulating_speculator(stock, regression_indicators, 0, 1),
    "mixed regulating speculator": lambda stock: regulating_speculator(stock, mixed_indicators, 0, 1),
}

## Price position volatility reduction
The absolute deviation from a mean as defined in the bollinger position experiment seems to be correlated to the future volatility. This strategy reduces the exposure to stocks, which deviated from the mean.

In [4]:
def volatility_reducer(stock, indicators):
    # a nanmean of an empty slice triggers a warning, but returns nan, which is correct
    warnings.simplefilter("ignore", category=RuntimeWarning)
    # get the indicator data
    indicator_values = stock[indicators]
    
    # calculate the mean, if possible
    mean_features = np.nanmean(np.abs(indicator_values), axis=1)
    
    # the rule causes a higher position in the stock, if the indicators are low (potentially underpriced)
    rule = (1 - np.where(np.isnan(mean_features), 0, mean_features))
    
    return np.maximum(rule, 0)



# define the indicators used for the different strategies
bollinger_indicators = indicators_bollinger = ["bollinger_position20_2", "bollinger_position50_2", "bollinger_position100_2", "bollinger_position200_2"]
horizontal_indicators = ["horizontal_position20", "horizontal_position50", "horizontal_position100", "horizontal_position200"]
regression_indicators = ["regression_position20", "regression_position50", "regression_position100", "regression_position200"]
mixed_indicators = ["bollinger_position50_2", "horizontal_position100", "regression_position100"]

# define the regulating strategies
volatility_reducer_strategies = {
    "bollinger volatility reducer": lambda stock: volatility_reducer(stock, bollinger_indicators),
    "support resistance volatility reducer": lambda stock: volatility_reducer(stock, horizontal_indicators),
    "regression position volatility reducer": lambda stock: volatility_reducer(stock, regression_indicators),
    "mixed regulating volatility reducer": lambda stock: volatility_reducer(stock, mixed_indicators)
}

## Relative strength balancer

The Relative Strength Index tries to detect time periods during which a stock is significantly overvalued or undervalued. A RSI above 70 is believed to signal an extreme upward move (overbought) and a value below 30 is believed to detect undervalued stocks. The RSI threshold transformation is already mapping those two cases to -1, 0 and 1, which makes the construction of the trading rule easier.


In [6]:
def rsi_average_balancer(stock, rsi):
    return np.where(stock[rsi] == -1, 1, 0)

rsi_strategies = {
    "rsi4 balancer": lambda stock: rsi_average_balancer(stock, "rsi_threshold4"),
    "rsi7 balancer": lambda stock: rsi_average_balancer(stock, "rsi_threshold7"),
    "rsi14 balancer": lambda stock: rsi_average_balancer(stock, "rsi_threshold14"),
    "rsi20 balancer": lambda stock: rsi_average_balancer(stock, "rsi_threshold20"),
}

## Moving average balancer
The moving average is an objective measure of fair price. If the price is above the moving average, it is objectively "high" and "low" otherwise. This rule sets a position in a stock, when the price is below a moving average.


In [8]:
def moving_average_balancer(stock, ma):
    return np.where(stock[ma] > 0, 1, 0)

ema_strategies = {
    "ema10 balancer": lambda stock: moving_average_balancer(stock, "ema10"),
    "ema20 balancer": lambda stock: moving_average_balancer(stock, "ema20"),
    "ema50 balancer": lambda stock: moving_average_balancer(stock, "ema50"),
    "ema100 balancer": lambda stock: moving_average_balancer(stock, "ema100"),
    "ema200 balancer": lambda stock: moving_average_balancer(stock, "ema200")
}

## SVM Classifier

In [13]:
# load the svm classifier
svm_indicators = ["sma10", "sma20", "sma50", "sma100", "lwma10", "lwma20", "lwma50", "lwma100", "lwma200",
              "ema10", "ema20", "ema50", "ema100", "rate_of_change20", "rate_of_change50", 
              "horizontal_position20", "horizontal_position50", "horizontal_position100",
              "regression_position20", "regression_position50", "regression_position100",
              "bollinger_position20_2", "bollinger_position50_2", "bollinger_position100_2"]

ftest_selected_indicators = ["macd12_26", "macd_signal12_26", "cci50", "horizontal_lower20", "horizontal_lower50", 
                             "ma_cross50_200", "horizontal_lower200", "regression_threshold20", 
                             "regression_threshold100", "chande100", "horizontal_lower100", "ma_cross20_50", 
                             "cci_threshold50", "lwma10", "regression_position20", "regression_upper20",
                             "regression_position100", "volatility10", "ema10", "aaron_oscillator40", 
                             "bollinger_lower20_2", "rsi20", "aaron_oscillator25", "horizontal_upper20",
                             "volatility20", "aaron_up40", "aaron_oscillator15", "aaron_down40", "sma10", 
                             "aaron_down25"]

svm_standard = joblib.load(STORAGE_PATH_MODELS.format("price_predictor_svm_standard.joblib"))
svm_fselected = joblib.load(STORAGE_PATH_MODELS.format("price_predictor_svm_fselected.joblib"))
svm_volatility = joblib.load(STORAGE_PATH_MODELS.format("volatility10_predictor_svm_standard.joblib"))

def svm_price_strategy(stock, svm, indicators):
    features = stock[indicators].fillna(0)
    return np.clip(svm.predict(features), 0, 1)

def svm_volatility_strategy(stock, svm, indicators):
    features = stock[indicators].fillna(0)
    return np.clip(-svm.predict(features), 0, 1)

svm_strategies = {
    "svm_standard": lambda stock: svm_price_strategy(stock, svm_standard, svm_indicators),
    "svm_f_selected": lambda stock: svm_price_strategy(stock, svm_fselected, ftest_selected_indicators),
    "svm_volatility": lambda stock: svm_volatility_strategy(stock, svm_volatility, svm_indicators)
}

## Neural network classifier

In [16]:
# small neural network for predicting the future price or volatility
class PriceHistoryNetwork(nn.Module):
    def __init__(self, input_neurons=24):
        super().__init__() 
        # define layers 
        self.fc1 = nn.Linear(input_neurons, 12)
        self.sigm1 = nn.Sigmoid()
        self.fc2 = nn.Linear(12, 8)
        self.sigm2 = nn.Sigmoid()
        self.fc3 = nn.Linear(8, 1)
        self.sigm3 = nn.Sigmoid()


    def forward(self, x):
        x = self.fc1(x)
        x = self.sigm1(x)
        x = self.fc2(x)
        x = self.sigm2(x)
        x = self.fc3(x)
        x = self.sigm3(x)
        return x

In [26]:
net_indicators = ["sma10", "sma20", "sma50", "sma100", "lwma10", "lwma20", "lwma50", "lwma100", "lwma200",
                  "ema10", "ema20", "ema50", "ema100", "rate_of_change20", "rate_of_change50", 
                  "horizontal_position20", "horizontal_position50", "horizontal_position100",
                  "regression_position20", "regression_position50", "regression_position100",
                  "bollinger_position20_2", "bollinger_position50_2", "bollinger_position100_2"]

ftest_selected_indicators = ["macd12_26", "macd_signal12_26", "cci50", "horizontal_lower20", "horizontal_lower50", 
                             "ma_cross50_200", "horizontal_lower200", "regression_threshold20", 
                             "regression_threshold100", "chande100", "horizontal_lower100", "ma_cross20_50", 
                             "cci_threshold50", "lwma10", "regression_position20", "regression_upper20",
                             "regression_position100", "volatility10", "ema10", "aaron_oscillator40", 
                             "bollinger_lower20_2", "rsi20", "aaron_oscillator25", "horizontal_upper20",
                             "volatility20", "aaron_up40", "aaron_oscillator15", "aaron_down40", "sma10", 
                             "aaron_down25"]

net_standard = PriceHistoryNetwork()
net_standard.load_state_dict(torch.load(STORAGE_PATH_MODELS.format("price_predictor_net_standard.pth")))
net_standard.eval()

net_all = PriceHistoryNetwork(len(all_indicators))
net_all.load_state_dict(torch.load(STORAGE_PATH_MODELS.format("price_predictor_net_all.pth")))
net_all.eval()

net_fselected = PriceHistoryNetwork(len(ftest_selected_indicators))
net_fselected.load_state_dict(torch.load(STORAGE_PATH_MODELS.format("price_predictor_net_fselected.pth")))
net_fselected.eval()

net_volatility = PriceHistoryNetwork()
net_volatility.load_state_dict(torch.load(STORAGE_PATH_MODELS.format("volatility10_predictor_net_standard.pth")))
net_volatility.eval()

def net_price_strategy(stock, net, indicators):
    features = torch.tensor(stock[indicators].fillna(0).values.astype(np.float32))
    classifications = net(features).detach().numpy()
    return np.maximum(np.sign(classifications - 0.50), 0)

def net_volatility_strategy(stock, net, indicators):
    features = torch.tensor(stock[indicators].fillna(0).values.astype(np.float32))
    classifications = net(features).detach().numpy()
    return -np.minimum(np.sign(classifications - 0.50), 0)

net_strategies = {
    "net_standard": lambda stock: net_price_strategy(stock, net_standard, net_indicators),
    "net_all": lambda stock: net_price_strategy(stock, net_all, all_indicators),
    "price_predictor_net_fselected": lambda stock: net_price_strategy(stock, net_fselected, ftest_selected_indicators),
    "net_volatilty": lambda stock: net_volatility_strategy(stock, net_volatility, net_indicators)
}

## Trend follower

The trend of a stock is defined to be positive, if a faster moving average is above the slower moving average.
This rule takes a position in a stock, if the trend is positive.

In [27]:
# define the trend follower strategy
def trend_follower(stock, trend):
    return np.where(np.isnan(stock[trend]), 0, np.where(stock[trend] > 0, 1, 0))

trend_follower_strategies = {
    "ma trend 20-50" : lambda stock: trend_follower(stock, "ma_trend20_50"),
    "ma trend 50-200" : lambda stock: trend_follower(stock, "ma_trend50_200")
}